In [1]:
import os
# manage directory-read operations using Posix paths
from pathlib import Path

In [2]:
_ROOT_DIR = '{0}/gcp/cbidmltsf'.format(os.getenv("HOME"))
_ROOT_DIR

'/home/jupyter/gcp/cbidmltsf'

In [3]:
_EQUIPMENT = 'CPE04015'

In [4]:
import numpy as np

In [5]:
from bokeh.plotting import figure, output_file, save

In [6]:
from sklearn.externals import joblib

/usr/local/lib/python3.5/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [7]:
import tensorflow as tf

In [9]:
_SAVED_MODEL_DIR = '/home/developer/gcp/cbidmltsf/dplstm/246401_073201_043201/export/exporter'

In [10]:
subdirs = [x for x in Path(_SAVED_MODEL_DIR).iterdir()
           if x.is_dir() and 'temp' not in str(x)]
_LATEST_SAVED_MODEL_DIR = str(sorted(subdirs)[-1])
_LATEST_SAVED_MODEL_DIR

'/home/developer/gcp/cbidmltsf/dplstm/246401_073201_043201/export/exporter/1580234058'

In [11]:
# build a prediction function
predict_fn = tf.contrib.predictor.from_saved_model(_LATEST_SAVED_MODEL_DIR)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from /home/developer/gcp/cbidmltsf/dplstm/246401_073201_043201/export/exporter/1580234058/variables/variables


In [12]:
# get the filename with the test set in tfrecord format
test_dataset_filename = '/home/developer/gcp/cbidmltsf/experiments/CPE04015_H_desbI_2017-04-01_00:00:00_2018-02-28_23:00:00_H024001001_D007024001_W004168001/test.tfrecord'

In [13]:
# get the test dataset as a tf.Data.Dataset
test_dataset = tf.data.TFRecordDataset(test_dataset_filename)

In [14]:
# build a simple iterator on the dataset to verify the structure of the serialized example
dataset = test_dataset.map(lambda row: row)
iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


In [16]:
# modified to run on TF 2.X
sess = tf.compat.v1.InteractiveSession()

In [17]:
value = sess.run(next_element)
print(value)

b"\n\x8f\x03\n)\n\x05oh_wd\x12 \x12\x1e\n\x1c\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80?\x00\x00\x00\x00\x00\x00\x00\x00\n)\n\x05daily\x12 \x12\x1e\n\x1cg\x8e\xa9>\x7f\xd4\xc7>\x10\xb3\x14?J\xfc\x0c?\x81'\x1b?)\xbd\x17?\xc8\x81\x0e?\n$\n\ttimestamp\x12\x17\n\x15\n\x132018-02-23 14:00:00\nm\n\x05oh_dh\x12d\x12b\n`\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\n\x12\n\x06target\x12\x08\x12\x06\n\x04\xce\xfb\xc5>\nn\n\x06hourly\x12d\x12b\n`\xc8\x81\x0e?\xbc\xd59?|c\xf0>\x04\x8d\x06?\xa5B\xac>]\x7f\xef>K=\x05?T\t\x0e?\t\xb9\xc7>\xcfs\xbd>\x07\xdc\x90>\x9cW\x84>\xf0\xb3f>\xfd\xc2v>;\xb2\x89>\x1c\x15\xb8>\xa7r\xb7>\xfck\xdd>\xc3\

In [18]:
# get predictions from the prediction function
predictions_list = []
with tf.compat.v1.Session() as sess:
    try:
        while True:
            example = sess.run(next_element)
            predictions_list.append(predict_fn({'example_bytes': example}))
    except:
        pass

In [19]:
predictions_list[:5]

[{'forecast': array([[0.52965707]], dtype=float32)},
 {'forecast': array([[0.54312223]], dtype=float32)},
 {'forecast': array([[0.5127039]], dtype=float32)},
 {'forecast': array([[0.42954084]], dtype=float32)},
 {'forecast': array([[0.3647977]], dtype=float32)}]

In [20]:
# reload scaler fitted model here
scaler = joblib.load('/home/developer/gcp/cbidmltsf/experiments/CPE04015_H_desbI_2017-04-01_00:00:00_2018-02-28_23:00:00_H024001001_D007024001_W004168001/scaler.save')

In [21]:
# move to array and re-scale
predictions = [p['forecast'][0][0] for p in predictions_list]
predictions = np.asarray(predictions)

In [22]:
# predictions array is ready
predictions[:5]

array([0.52965707, 0.54312223, 0.5127039 , 0.42954084, 0.3647977 ],
      dtype=float32)

In [23]:
# inverse scale predictions
predictions = scaler.inverse_transform(predictions.reshape(-1, 1))

In [24]:
predictions = np.squeeze(predictions)

In [25]:
# re-scaled predictions array is ready for plotting
predictions[:5]

array([8.511033 , 8.727404 , 8.238613 , 6.9022703, 5.861916 ],
      dtype=float32)

In [26]:
# get target values and timestamps for analysis and plotting
# a new _parse_dataset_function is required

In [27]:
# define read features dictionary
read_features = {
    'hourly': tf.VarLenFeature(dtype=tf.float32),
    'daily': tf.VarLenFeature(dtype=tf.float32),
    'weekly': tf.VarLenFeature(dtype=tf.float32),
    'target': tf.VarLenFeature(dtype=tf.float32),
    'oh_wd': tf.VarLenFeature(dtype=tf.float32),
    'oh_dh': tf.VarLenFeature(dtype=tf.float32),
    'timestamp': tf.VarLenFeature(dtype=tf.string) 
}

In [28]:
# store the objective shapes for reshaping tensors in a dictionary
objective_shapes = {
    'hourly': [24, 1],
    'daily': [7, 1],
    'weekly': [4, 1],
    'target': [1, 1],
    'oh_wd': [7, 1], # Monday to Sunday
    'oh_dh': [24, 1], # midnight to 23:00
    'timestamp': [1, 1]
}

In [29]:
def _parse_dataset_function(example_proto, objective_shapes):
    # parse the input tf.Example proto using the dictionary above
    row = tf.io.parse_single_example(example_proto, read_features)
    # pass objective shape as a list of lists [hourly_shape, daily_shape, weekly_shape]
    hourly = tf.reshape(row['hourly'].values, objective_shapes['hourly'])
    daily = tf.reshape(row['daily'].values, objective_shapes['daily'])
    weekly = tf.reshape(row['weekly'].values, objective_shapes['weekly'])
    target = tf.reshape(row['target'].values, objective_shapes['target'])
    oh_wd = tf.reshape(row['oh_wd'].values, objective_shapes['oh_wd'])
    oh_dh = tf.reshape(row['oh_dh'].values, objective_shapes['oh_dh'])
    timestamp = tf.reshape(row['timestamp'].values, objective_shapes['timestamp'])
    
    return {'hourly': hourly,
             'daily': daily,
            'weekly': weekly,
            'target': target,
            'oh_wd': oh_wd,
            'oh_dh': oh_dh,
            'timestamp': timestamp}

In [30]:
# test_dataset was previously acquired from tfrecord file, use it again to build arrays for targets and timestamps

In [32]:
dataset = test_dataset.map(lambda row: _parse_dataset_function(row, objective_shapes))
iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()

In [33]:
from pandas import to_datetime

In [34]:
timestamps_list = []
targets_list = []
with tf.compat.v1.Session() as sess:
    try:
        while True:
            item = sess.run(next_element)
            # get the timestamp, then the scalar value, then the string value, then convert it to datetime
            timestamps_list.append(to_datetime(item['timestamp'][0][0].decode()))
            # get the target, then the scalar value
            targets_list.append(item['target'][0][0])
    except:
        pass

In [35]:
timestamps_list[:5]

[Timestamp('2018-02-23 14:00:00'),
 Timestamp('2018-02-23 15:00:00'),
 Timestamp('2018-02-23 16:00:00'),
 Timestamp('2018-02-23 17:00:00'),
 Timestamp('2018-02-23 18:00:00')]

In [37]:
timestamps = np.asarray(timestamps_list)

In [38]:
targets = np.asarray(targets_list)

In [39]:
targets = scaler.inverse_transform(targets.reshape(-1, 1))

In [40]:
targets = np.squeeze(targets)

In [41]:
targets[:5]

array([6.2136497, 8.161867 , 6.7520003, 5.244833 , 7.2472496],
      dtype=float32)

In [42]:
# everything is ready now for plotting

In [46]:
predictions_fig = figure(title='Predicted Current Imbalance for ' + _EQUIPMENT,
                         background_fill_color='#E8DDCB',
                         plot_width=1800, plot_height=450, x_axis_type='datetime')

In [47]:
predictions_fig.line(timestamps, targets, line_color='red',
                     line_width=1, alpha=0.7, legend_label='Actual')

GlyphRenderer(id='1126', ...)

In [48]:
predictions_fig.line(timestamps, predictions, line_color='blue',
                     line_width=1, alpha=0.7, legend_label='Predicted')

GlyphRenderer(id='1153', ...)

In [49]:
predictions_fig.legend.location = "top_right"
predictions_fig.legend.background_fill_color = "darkgrey"
predictions_fig.xaxis.axis_label = 'Timestamp'
predictions_fig.yaxis.axis_label = 'Current Imbalance [%]'

In [ ]:
# temporarily save to jupyter to avoid permissions conflicts

In [52]:
output_file('/home/jupyter/gcp/cbidmltsf/dplstm/predictions.html', title=_EQUIPMENT)
save(predictions_fig)

'/home/jupyter/gcp/cbidmltsf/dplstm/predictions.html'

In [ ]:
# everything works fine, so far, let's transfer the code to a batch script